# Machine Learning Part
* goal of this notebook is to develop code for the machine learning portion
    * dataset loading
    * model
    * evaluation metrics
* code should be added into qdanalysis and refactored aferwards

In [1]:
import tensorflow as tf
from tensorflow import keras

from qdanalysis.models import adapt_resnet50

import pathlib

2024-02-15 02:49:05.450846: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-15 02:49:05.481565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 02:49:05.481588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 02:49:05.482397: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-15 02:49:05.487501: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_dir = './output/cerug_test_processing/CN'
test_dir = './output/cerug_test_processing/EN'

In [4]:
label_mode = 'categorical'
#shape image will be resized to (figure out ideal values later)
input_shape = (128, 128, 3)
seed = 42
validation_split = 0.2

#due to nature of data, the test and train splits are defined beforehand
train_data = keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                              label_mode=label_mode, 
                                                              image_size=input_shape[:2], 
                                                              seed=seed, 
                                                              validation_split=validation_split, 
                                                              subset='training')
writer_classes = train_data.class_names

Found 12345 files belonging to 3 classes.
Using 9876 files for training.


2024-02-15 02:49:08.724779: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21720 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:65:00.0, compute capability: 7.5


In [5]:
writer_classes

['Writer0101', 'Writer0202', 'Writer0303']

In [6]:
validation_data = keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                   label_mode=label_mode,
                                                                   image_size=input_shape[:2],
                                                                   seed=seed,
                                                                   validation_split=validation_split,
                                                                   subset='validation')

Found 12345 files belonging to 3 classes.
Using 2469 files for validation.


In [7]:
model = adapt_resnet50(input_shape, len(writer_classes))

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1000)              2049000   
                                                                 
 dense_1 (Dense)             (None, 3)                 3003      
                                                                 
Total params: 25639715 (97.81 MB)
Trainable params: 2052003 (7.83 MB)
Non-trainable params: 23587712 (89.98 MB)
_______________

In [8]:
# optimizer and loss taken from https://stackoverflow.com/questions/71704268/using-tf-keras-utils-image-dataset-from-directory-with-label-list
optimizer = tf.keras.optimizers.Nadam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,
    name='Nadam'
)

lossfn = tf.keras.losses.CategoricalCrossentropy()

metrics = [
    keras.metrics.CategoricalAccuracy(),
    keras.metrics.Precision(),
    keras.metrics.Recall()
]

model.compile(optimizer=optimizer, loss=lossfn, metrics=metrics)

In [9]:
epochs = 10
history = model.fit(train_data, epochs=epochs, validation_data=validation_data)

Epoch 1/10


2024-02-15 02:42:04.789773: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-15 02:42:05.871623: I external/local_xla/xla/service/service.cc:168] XLA service 0x9807f70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-15 02:42:05.871657: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN RTX, Compute Capability 7.5
2024-02-15 02:42:05.887468: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1707993726.025951  322652 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


309/309 [==============================] - 12s 28ms/step - loss: 1.1000 - categorical_accuracy: 0.4184 - precision: 0.4694 - recall: 0.1882 - val_loss: 1.0395 - val_categorical_accuracy: 0.4500 - val_precision: 0.5548 - val_recall: 0.2175
Epoch 2/10
309/309 [==============================] - 7s 21ms/step - loss: 0.9973 - categorical_accuracy: 0.5042 - precision: 0.5981 - recall: 0.2729 - val_loss: 0.9968 - val_categorical_accuracy: 0.4953 - val_precision: 0.5893 - val_recall: 0.2714
Epoch 3/10
309/309 [==============================] - 7s 21ms/step - loss: 0.9546 - categorical_accuracy: 0.5398 - precision: 0.6337 - recall: 0.3169 - val_loss: 0.9696 - val_categorical_accuracy: 0.5233 - val_precision: 0.6173 - val_recall: 0.3038
Epoch 4/10
309/309 [==============================] - 7s 21ms/step - loss: 0.9232 - categorical_accuracy: 0.5650 - precision: 0.6609 - recall: 0.3500 - val_loss: 0.9541 - val_categorical_accuracy: 0.5379 - val_precision: 0.6279 - val_recall: 0.3260
Epoch 5/10
309

In [9]:
#due to nature of data, the test and train splits are defined beforehand
test_data = keras.preprocessing.image_dataset_from_directory(test_dir, 
                                                              label_mode=label_mode, 
                                                              image_size=input_shape[:2], 
                                                              seed=seed)
writer_classes_test = test_data.class_names

Found 3636 files belonging to 3 classes.


In [10]:
writer_classes

['Writer0101', 'Writer0202', 'Writer0303']

In [11]:
writer_classes_test

['Writer0101', 'Writer0202', 'Writer0303']

In [12]:
results = model.evaluate(test_data)

2024-02-15 02:49:50.204165: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


114/114 [==============================] - 4s 22ms/step - loss: 2.3899 - categorical_accuracy: 0.3232 - precision: 0.3245 - recall: 0.3218
